In [1]:
import pandas as pd
import numpy as np
from itertools import chain, combinations

from sklearn.model_selection import cross_val_score
import sklearn.model_selection
from sklearn import svm
import sklearn.metrics
from sklearn.model_selection import train_test_split

In [2]:
# Transforme une liste de fichiers en dictionnaire de fichiers avec les même proportion d'embbeddings
def listToDicSameProp(lst):
    return {elt: 1./len(lst) for elt in lst}

# Créer toute les combinaisons
def listToDicCombiSameProp(lst, minLen=1, maxLen=-1):
    if maxLen == -1:
        maxLen = len(lst)
    return [listToDicSameProp(x) for x in chain.from_iterable(combinations(lst, r) for r in range(minLen, maxLen+1))]

# Fusion de listes
def listMerge2(lst1, lst2, prop1):
    return [{elt1: prop1, elt2: 1.-prop1} for elt1 in lst1 for elt2 in lst2]

def eltsGradient(elt1, elt2):
    return [{elt1: x, elt2: 1.-x} for x in np.arange(.1, 1., 0.1)]

In [9]:
# Liste des tests a effectuer
# Pour chaque test : un dictionnaire fichier embbedding -> ratio de ce fichier
filenamesLists = [{"messages_equilibre_ok.npy": 1.}]
filenamesLists

[{'messages_equilibre_ok.npy': 1.0}]

In [10]:
def testEmbd(filenames):
    # Chargement des données
    X = np.ndarray((0,0))
    y = []
    for embdFile, proportion in filenames.items():
        dataFile = np.load(embdFile, allow_pickle=True)
        if len(X) == 0:
            X = proportion*dataFile
        else:
            X += proportion*dataFile
    y = np.load("label.npy")

    # Tests
    cv = sklearn.model_selection.ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)
    clf = svm.SVC(kernel='rbf', C=1, random_state=42)
    results = cross_val_score(clf, X, y, cv=cv, scoring="f1")
    print("F-score de la validation croisée : ", results)
    print("Moyenne des F-score", np.mean(results))
    return np.mean(results)

In [11]:
resultsGlobal = {str(filenames): testEmbd(filenames) for filenames in filenamesLists}
resultsGlobal

F-score de la validation croisée :  [0.88778055 0.84634761 0.86893204 0.8973747  0.87       0.86436782
 0.85714286 0.86934673 0.85714286 0.87407407]
Moyenne des F-score 0.8692509234307005


{"{'messages_equilibre_ok.npy': 1.0}": 0.8692509234307005}